In [2]:
%pwd

'r:\\TAZMIC\\src\\research'

In [ ]:
import os
os.chdir("../")

In [33]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from model import llm, embedding_model
from langchain_chroma import Chroma
import os

loader = UnstructuredMarkdownLoader(
    "R:/TAZMIC/artifacts/research_papers/biology/content.md",
    mode="single",
    strategy="fast",
)

In [34]:
loaded_docs = loader.load()

In [35]:
pages = loaded_docs

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

pages_split = text_splitter.split_documents(pages) 

In [38]:
persist_directory = r"R:/TAZMIC/artifacts/Vector_databases/biology"
collection_name = "biology"

if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

In [ ]:
try:
    vectorstore = Chroma.from_documents(
        documents=pages_split,
        embedding=embedding_model,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    print(f"Created ChromaDB vector store!")
    
except Exception as e:
    print(f"Error setting up ChromaDB: {str(e)}")
    raise


# Now we create our retriever 
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20}
)